# ***Libraries***

# Non-Streaming Methods
We pick two non-streaming methods of our choice as a baseline (e.g., Isolation Forest and a DNN
method). We run the methods on the files of the dictionary we created in an eariler step. In the no streaming setting we use the entire generated files as input. These methods will serve as our offline baselines.
